In [38]:
import tiktoken

In [39]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-16k-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [40]:
input_messages = [
    {
      "role": "system",
      "content": "Действуй как профессиональный рекрутер. Твоя задача проанализировать резюме кандидата, которое я тебе пришлю и сказать, подходит он на эту вакансию или нет."
    },
    {
      "role": "user",
      "content": ""
    },
    {
      "role": "user",
      "content": "Напиши ответ в формате JSON, без лишнего текста: \nЕсли кандидат подходит: {\"Подходит ли кандидат  на вакансию\" : ..., \"Комментарий\":...}\nНазвание вакансии: HR-аналитик\nЧем занимается компания:  Фриланса в интернете\nТребования к кандидату: \n-Опыт на HR-аналитика. \n-Способность глубоко погружаться в детали. \n-Быстро находить взаимосвязи и варианты решения. \n-Отличная коммуникабельность + Позитивный настрой. \n-Чем предстоит заниматься на данной вакансии: \n-Выбрать ключевые метрики и постоянно отслеживать их\n-Выявлять проблемы HR-отдела, ухудшающие метрики\n-Составлять план-карту улучшений HR-отдела\n-Анализировать выявленные проблемы, разрабатывать обоснованные решения этих проблем.\nУсловия работы:\n-Удаленная работа по комфортному для вас графику (от 30 часов в неделю)\n-Почасовая ставка 1 500 руб в час\n-Выплаты каждые две недели.\n-Логично выстроенная система коммуникаций"
    },
    {
      "role": "assistant",
      "content": "Хорошо, ожидаю вашего резюме для анализа."
    },
    {
      "role": "user",
      "content": "Аналитик данных\nСпециализации:\n— Аналитик\n— Бизнес-аналитик\n— Продуктовый аналитик\n— BI-аналитик, аналитик данных\nЗанятость: полная занятость, частичная занятость, проектная работа\nГрафик работы: полный день, гибкий график, удаленная работа\nЖелательное время в пути до работы: не более часа\nОпыт работы — 8 лет 9 месяцев\nЯнварь 2022 —\nнастоящее время\n1 год 5 месяцев\nУниверситет ИТМО\nСанкт-Петербург\nОбразовательные учреждения\n• Вуз, ссуз колледж, ПТУ\nАналитик центра карьеры обучающихся\n- Разработчик и реализатор концепции создания кадрового агентства на базе университета:\nсоздала базу резюме студентов в CRM, автоматизировав поиск кандидатов под запросы\nработодателя. Автоматизировала процесс подачи заявок от студентов и работодателей\n- Разработчик и реализатор проекта анализа рынка труда для руководителей образовательных\nпрограмм: провожу исследования рынка труда с использованием API hh.ru\n(https://github.com/eshamanova/hh_analytics)\n- Веду телеграм-канал Центра карьеры для иностранных студентов\n- Готовлю аналитические отчёты о местах прохождения практической подготовки студентов,\nиспользуя Excel и Power BI\n- Придумала и разработала карьерную настольную игру для студентов\n- Разработка технических заданий для оптимизации и цифровизации процессов в Центре\nкарьеры\n- Организатор и спикер на мероприятиях, в том числе на английском языке\n- Менеджер по рекрутингу в рамках проекта \"ITMO.Advising\" (18 отобранных и\nтрудоустроенных специалистов)\nДекабрь 2021 —\nДекабрь 2022\n1 год 1 месяц\nЯндекс.Практикум (учебный опыт)\nСтудент – Аналитик данных\nРешение практических задач в области анализа данных и машинного обучения\n(https://github.com/eshamanova/MyYandexPracticumPortfolio):\n- Анализ убытков мобильного приложения ProcrastinatePRO. Описание: анализ данных от\nкомпании; расчет различных метрик, таких как LTV, CAC, Retention rate, DAU, WAU, MAU и т.д;\nРезюме обновлено 11 мая 2023 в 11:43\nприменение когортного анализа; выводы для повышения прибыльности компании.\nТехнологии: Pandas, Matplotlib, Seaborn, когортный анализ, юнит-экономика, продуктовые\nметрики\n- Прогнозирование вероятности оттока пользователей для фитнес-центров. Описание:\nприменение машинного обучения для предсказания вероятности оттока клиентов; создание\nтипичных профилей пользователей и определение их ключевых характеристик; выделение\nнаиболее значимых признаков, влияющих на вероятность оттока клиентов. Технологии:\nPandas, Scikit-learn, Matplotlib, Seaborn, машинное обучение, классификация, кластеризация\n- Сегментация покупателей интернет-магазина товаров для дома и быта (Выпускной проект).\nОписание: исследовательский анализ данных показателей интернет-магазина товаров для\nдома, сегментация покупателей на основе истории их покупок, формулировка и проверка\nстатистических гипотез, формулировка рекомендаций по разработке персонализированных\nпредложений для покупателей, реализация которых может повысить уровень продаж,\nподготовка презентации и дашборда. Технологии: Pandas, Matplotlib, Seaborn, Tableau,\nпродуктовые метрики, сегментация, формулировка и проверка гипотез, формулировка\nпредложений для увеличения прибыли, построение дашбордов\nЯнварь 2021 —\nЯнварь 2022\n1 год 1 месяц\nУниверситет ИТМО\nСанкт-Петербург\nОбразовательные учреждения\n• Вуз, ссуз колледж, ПТУ\nМенеджер студенческого офиса\n- Изготовление различных видов справок для студентов на русском и английском языках\n- Подготовка ответов на официальные запросы государственных структур\n- Регистрация входящей и исходящей документации\n- Консультирование студентов по электронной почте, телефону и лично (на русском и\nанглийском языках)\nОктябрь 2019 —\nАвгуст 2020\n11 месяцев\nНациональный исследовательский университет «Высшая школа\nэкономики»\nСанкт-Петербург, www.hse.ru\nОбразовательные учреждения\n• Вуз, ссуз колледж, ПТУ\nДиспетчер факультета\n- Составление расписания занятий, сессий, дополнительных сессий в соответствии с учебным\nпланом и требованиями, изложенными в нормативной документации\n- Внесение расписания в систему электронного расписания, оперативное внесение изменений,\nконтроль\n- Взаимодействие с преподавателями, студентами, другими подразделениями университета (в\nтом числе на английском языке): деловые переговоры, деловая переписка\n- Бронирование аудиторий в системе электронно расписания\n- Информирование студентов по электронным каналам\n- Администрирование корпоративной учётной записи Zoom, осуществление контроля и\nтехнической поддержки во время проведения дистанционных занятий\nМай 2014 —\nОктябрь 2019\n5 лет 6 месяцев\nАО Лимб\nСанкт-Петербург\nСтроительство, недвижимость, эксплуатация, проектирование\n• Архитектура, проектирование\nИнженер-картограф\nШаманова Екатерина • Резюме обновлено 11 мая 2023 в 11:43\nПодготовка землеустроительной документации\nОбразование\nВысшее\n2013 Санкт-Петербургский государственный университет,\nСанкт-Петербург\nФакультет географии и геоэкологии, География (географ)\nПовышение квалификации, курсы\n2022 Аналитик данных\nЯндекс.Практикум\n2021 Soft Skills\nУниверситет ИТМО\n2019 Переводчик в сфере профессиональной коммуникации\nСПбГУ\nКлючевые навыки\nЗнание языков Русский — Родной\nАнглийский — C1 — Продвинутый\nНавыки Работа с большим объемом информации Работа в команде\nАнглийский язык ГИС Письменный перевод Грамотность\nДеловая коммуникация Деловая переписка Python SQL MS Excel \n Pandas Tableau A/B тесты JUPYTER Проверка статистических гипотез\nИсследовательский анализ данных Предобработка данных\n A/B тестирование Когортный анализ Plotly Визуализация данных\n PowerBI ChatGPT Stable Diffusion MidJourney Google Docs \nДополнительная информация\nОбо мне На текущем месте работы выступаю в роли идейного вдохновителя, разработчика и\nреализатора нескольких инновационных проектов как внутри своего подразделения, так и\nв составе внутриуниверситетской команды.\nБольше года регулярно использую Python, Excel и Power BI для анализа и визуализации\nданных.\nЛюблю учиться и придумывать новое. Умею работать в команде и ценю возможность\nобмена опытом. Всегда ответственно отношусь к исполнению своих обязанностей. Обладаю\nвысоким уровнем эмпатии. Имею опыт работы как в бизнесе, так и в образовании,\nблагодаря чему научилась находить общий язык с разными людьми и быстро\nадаптироваться"
    }]

In [41]:
input_token = num_tokens_from_messages(input_messages, "gpt-3.5-turbo-16k-0613")
input_token

3420

In [42]:
output_message = [{
      "role": "assistant",
      "content": "{\n   \"Подходит ли кандидат  на вакансию\": true,\n   \"Комментарий\": \"Кандидат имеет опыт работы с анализом данных и различными аналитическими инструментами, такими как Python, Excel и Power BI. Также обладает навыками работы с SQL и проведения статистического анализа. Успешно выполняла проекты по анализу данных в разных сферах применения. Обладает высоким уровнем коммуникабельности и способностью работать в команде. Опыт рекрутинга и разработки процессов в области карьеры также является преимуществом для данной вакансии в HR-аналитике.\"\n}"
    }]

In [43]:
output_token = num_tokens_from_messages(output_message, "gpt-3.5-turbo-16k-0613")
output_token

235

In [44]:
input_price = 0.003
output_price = 0.004
total_price = (input_token * input_price + output_price * output_token) / 1000
total_price

0.0112